In [41]:
#!pip install torch==1.2.0+cpu torchvision==0.4.0+cpu -f https://download.pytorch.org/whl/torch_stable.html
#!pip install plotly
import time
import copy
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.optim as optim
import torch.nn.functional as F
from plotly import tools
from plotly.graph_objs import *
from plotly.offline import init_notebook_mode, iplot, iplot_mpl


In [42]:
data = pd.read_csv('MSFT.csv')
data['Date'] = pd.to_datetime(data['Date'])
data = data.set_index('Date')
print(data.index.min(), data.index.max())
data.head()

2010-01-04 00:00:00 2019-07-05 00:00:00


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2010-01-04,30.62,31.10,30.59,30.95,24.53,38409100
2010-01-05,30.85,31.10,30.64,30.96,24.53,49749600
2010-01-06,30.88,31.08,30.52,30.77,24.38,58182400
2010-01-07,30.63,30.70,30.19,30.45,24.13,50559700
2010-01-08,30.28,30.88,30.24,30.66,24.30,51197400


In [43]:
date_split = '2018-01-01'
train = data[:date_split]
test = data[date_split:]
len(train),len(test)

(2013, 379)

In [44]:
def plot_train_test(train, test, date_split):
    
    data = [
        Candlestick(x=train.index, open=train['Open'], high=train['High'], low=train['Low'], close=train['Close'], name='train'),
        Candlestick(x=test.index, open=test['Open'], high=test['High'], low=test['Low'], close=test['Close'], name='test')
    ]
    layout = {
         'shapes': [
             {'x0': date_split, 'x1': date_split, 'y0': 0, 'y1': 1, 'xref': 'x', 'yref': 'paper', 'line': {'color': 'rgb(0,0,0)', 'width': 1}}
         ],
        'annotations': [
            {'x': date_split, 'y': 1.0, 'xref': 'x', 'yref': 'paper', 'showarrow': False, 'xanchor': 'left', 'text': ' test data'},
            {'x': date_split, 'y': 1.0, 'xref': 'x', 'yref': 'paper', 'showarrow': False, 'xanchor': 'right', 'text': 'train data '}
        ]
    }
    figure = Figure(data=data, layout=layout)
    iplot(figure)

In [45]:
plot_train_test(train, test, date_split)

In [46]:
class Environment1:
    
    def __init__(self, data, history_t=90):
        self.data = data
        self.history_t = history_t
        self.reset()
        
    def reset(self):
        self.t = 0
        self.done = False
        self.profits = 0
        self.positions = []
        self.position_value = 0
        self.history = [0 for _ in range(self.history_t)]
        return [self.position_value] + self.history # obs
    
    def step(self, act):
        reward = 0
        
        # act = 0: stay, 1: buy, 2: sell
        if act == 1:
            self.positions.append(self.data.iloc[self.t, :]['Close'])
        elif act == 2: # sell
            if len(self.positions) == 0:
                reward = -1
            else:
                profits = 0
                for p in self.positions:
                    profits += (self.data.iloc[self.t, :]['Close'] - p)
                reward += profits
                self.profits += profits
                self.positions = []
        
        # set next time
        self.t += 1
        
        self.position_value = 0
        for p in self.positions:
            self.position_value += (self.data.iloc[self.t, :]['Close'] - p)
        self.history.pop(0)
        self.history.append(self.data.iloc[self.t, :]['Close'] - self.data.iloc[(self.t-1), :]['Close'])
        if (self.t==len(self.data)-1):
            self.done=True
        # clipping reward
        if reward > 0:
            reward = 1
        elif reward < 0:
            reward = -1
        #print ("t={%d}, done={%str}"%(self.t,self.done))
        return [self.position_value] + self.history, reward, self.done # obs, reward, done
    

In [47]:
env = Environment1(train)
print(env.reset())
for _ in range(3):
    pact = np.random.randint(3)
    print (pact)
    print(env.step(pact))

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
1
([0.010000000000001563, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.010000000000001563], 0, False)
2
([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.010000000000001563, -0.19000000000000128], 1, False)
0
([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [48]:
#def train_dqn(env):
    
class Q_Network(nn.Module):
        
    def __init__(self,obs_len,hidden_size,actions_n):
            
        super(Q_Network,self).__init__()
            
        self.fc_val = nn.Sequential(
            nn.Linear(obs_len, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, actions_n)
        )
        
        
    def forward(self,x):
        h =  self.fc_val(x)
        return (h)
            

In [49]:
hidden_size=100
input_size=env.history_t+1
output_size=3
USE_CUDA = False
LR = 0.001

In [50]:
Q = Q_Network(input_size, hidden_size, output_size)

In [51]:
Q_ast = copy.deepcopy(Q)

In [52]:
if USE_CUDA:
    Q = Q.cuda()
loss_function = nn.MSELoss()
optimizer = optim.Adam(list(Q.parameters()), lr=LR)

In [53]:
epoch_num = 50 #50
step_max = len(env.data)-1
memory_size = 200
batch_size = 50
gamma = 0.97

In [54]:
obs, reward, done = env.step(5)

In [55]:
memory = []
total_step = 0
total_rewards = []
total_losses = []
epsilon = 1.0
epsilon_decrease = 1e-3
epsilon_min = 0.1
start_reduce_epsilon = 200
train_freq = 10
update_q_freq = 20
gamma = 0.97
show_log_freq = 5

In [56]:
start = time.time()
for epoch in range(epoch_num):

    pobs = env.reset()
    step = 0
    done = False
    total_reward = 0
    total_loss = 0

    while not done and step < step_max:

        # select act
        pact = np.random.randint(3)
        if np.random.rand() > epsilon:
            pact = Q(torch.from_numpy(np.array(pobs, dtype=np.float32).reshape(1, -1)))
            pact = np.argmax(pact.data)
            pact = pact.numpy()

        # act
        obs, reward, done = env.step(pact)

        # add memory
        memory.append((pobs, pact, reward, obs, done))
        if len(memory) > memory_size:
            memory.pop(0)

        # train or update q
        if len(memory) == memory_size:
            if total_step % train_freq == 0:
                shuffled_memory = np.random.permutation(memory)
                memory_idx = range(len(shuffled_memory))
                for i in memory_idx[::batch_size]:
                    batch = np.array(shuffled_memory[i:i+batch_size])
                    b_pobs = np.array(batch[:, 0].tolist(), dtype=np.float32).reshape(batch_size, -1)
                    b_pact = np.array(batch[:, 1].tolist(), dtype=np.int32)
                    b_reward = np.array(batch[:, 2].tolist(), dtype=np.int32)
                    b_obs = np.array(batch[:, 3].tolist(), dtype=np.float32).reshape(batch_size, -1)
                    b_done = np.array(batch[:, 4].tolist(), dtype=np.bool)

                    q = Q(torch.from_numpy(b_pobs))
                    q_ = Q_ast(torch.from_numpy(b_obs))
                    maxq = np.max(q_.data.numpy(),axis=1)
                    target = copy.deepcopy(q.data)
                    for j in range(batch_size):
                        target[j, b_pact[j]] = b_reward[j]+gamma*maxq[j]*(not b_done[j])
                    Q.zero_grad()
                    loss = loss_function(q, target)
                    total_loss += loss.data.item()
                    loss.backward()
                    optimizer.step()
                    
            if total_step % update_q_freq == 0:
                Q_ast = copy.deepcopy(Q)
                
            # epsilon
            if epsilon > epsilon_min and total_step > start_reduce_epsilon:
                epsilon -= epsilon_decrease

            # next step
            total_reward += reward
            pobs = obs
            step += 1
            total_step += 1

        total_rewards.append(total_reward)
        total_losses.append(total_loss)

        if (epoch+1) % show_log_freq == 0:
            log_reward = sum(total_rewards[((epoch+1)-show_log_freq):])/show_log_freq
            log_loss = sum(total_losses[((epoch+1)-show_log_freq):])/show_log_freq
            elapsed_time = time.time()-start
            print('\t'.join(map(str, [epoch+1, epsilon, total_step, log_reward, log_loss, elapsed_time])))
            start = time.time()
            
#return Q, total_losses, total_rewards

5	0.0999999999999992	7850	-22166.6	307038.819276412	73.83429074287415
5	0.0999999999999992	7851	-22166.6	307038.87896759453	0.04297161102294922
5	0.0999999999999992	7852	-22166.6	307038.93865877704	0.007192850112915039
5	0.0999999999999992	7853	-22166.6	307038.9983499596	0.009563684463500977
5	0.0999999999999992	7854	-22166.6	307039.0580411421	0.011249303817749023
5	0.0999999999999992	7855	-22166.6	307039.11773232464	0.00603485107421875
5	0.0999999999999992	7856	-22166.6	307039.17742350715	0.007978677749633789
5	0.0999999999999992	7857	-22166.6	307039.23711468966	0.011379480361938477
5	0.0999999999999992	7858	-22166.6	307039.29680587223	0.007082223892211914
5	0.0999999999999992	7859	-22166.6	307039.35649705474	0.009138345718383789
5	0.0999999999999992	7860	-22166.6	307039.41618823726	0.007909297943115234
5	0.0999999999999992	7861	-22166.6	307039.5256979192	0.06159377098083496
5	0.0999999999999992	7862	-22166.6	307039.63520760107	0.008975744247436523
5	0.0999999999999992	7863	-22166.6	3

5	0.0999999999999992	7960	-22157.2	307065.928765432	0.010308027267456055
5	0.0999999999999992	7961	-22157.8	307066.3589268589	0.051912784576416016
5	0.0999999999999992	7962	-22158.4	307066.7890882859	0.007978200912475586
5	0.0999999999999992	7963	-22159.2	307067.2192497129	0.008157014846801758
5	0.0999999999999992	7964	-22160.0	307067.64941113984	0.009979724884033203
5	0.0999999999999992	7965	-22160.6	307068.0795725668	0.008996248245239258
5	0.0999999999999992	7966	-22161.2	307068.50973399373	0.008322715759277344
5	0.0999999999999992	7967	-22161.8	307068.9398954207	0.0060269832611083984
5	0.0999999999999992	7968	-22162.4	307069.3700568477	0.008960723876953125
5	0.0999999999999992	7969	-22163.0	307069.80021827464	0.010245561599731445
5	0.0999999999999992	7970	-22163.6	307070.2303797016	0.011970758438110352
5	0.0999999999999992	7971	-22164.2	307070.6896885714	0.048052072525024414
5	0.0999999999999992	7972	-22164.8	307071.1489974411	0.010031461715698242
5	0.0999999999999992	7973	-22165.4	

5	0.0999999999999992	8070	-22236.2	307131.7286328869	0.012993097305297852
5	0.0999999999999992	8071	-22236.6	307132.522097645	0.05640125274658203
5	0.0999999999999992	8072	-22237.0	307133.315562403	0.013684749603271484
5	0.0999999999999992	8073	-22237.4	307134.1090271611	0.018012285232543945
5	0.0999999999999992	8074	-22238.0	307134.9024919191	0.014645576477050781
5	0.0999999999999992	8075	-22238.6	307135.6959566772	0.010973930358886719
5	0.0999999999999992	8076	-22239.2	307136.4894214353	0.010699748992919922
5	0.0999999999999992	8077	-22239.8	307137.2828861933	0.013087749481201172
5	0.0999999999999992	8078	-22240.4	307138.0763509514	0.015069723129272461
5	0.0999999999999992	8079	-22241.0	307138.8698157094	0.011962652206420898
5	0.0999999999999992	8080	-22241.6	307139.6632804675	0.014425516128540039
5	0.0999999999999992	8081	-22242.0	307140.4862308189	0.06331014633178711
5	0.0999999999999992	8082	-22242.4	307141.30918117036	0.011044740676879883
5	0.0999999999999992	8083	-22242.8	307142

5	0.0999999999999992	8180	-22258.4	307234.9261840461	0.013052701950073242
5	0.0999999999999992	8181	-22258.2	307236.024622803	0.051738739013671875
5	0.0999999999999992	8182	-22258.0	307237.1230615599	0.009010791778564453
5	0.0999999999999992	8183	-22257.8	307238.2215003168	0.009163141250610352
5	0.0999999999999992	8184	-22257.6	307239.31993907376	0.015137910842895508
5	0.0999999999999992	8185	-22257.4	307240.41837783065	0.014270305633544922
5	0.0999999999999992	8186	-22257.2	307241.51681658754	0.013504981994628906
5	0.0999999999999992	8187	-22257.0	307242.6152553445	0.01558375358581543
5	0.0999999999999992	8188	-22256.8	307243.7136941014	0.009971141815185547
5	0.0999999999999992	8189	-22256.6	307244.8121328583	0.01499629020690918
5	0.0999999999999992	8190	-22256.4	307245.9105716152	0.01581859588623047
5	0.0999999999999992	8191	-22256.2	307247.028363215	0.06908512115478516
5	0.0999999999999992	8192	-22256.0	307248.14615481475	0.011085748672485352
5	0.0999999999999992	8193	-22256.0	30724

5	0.0999999999999992	8290	-22241.8	307369.342743109	0.013936519622802734
5	0.0999999999999992	8291	-22241.8	307370.78052951547	0.054622650146484375
5	0.0999999999999992	8292	-22242.0	307372.2183159219	0.00897836685180664
5	0.0999999999999992	8293	-22242.4	307373.6561023284	0.007996082305908203
5	0.0999999999999992	8294	-22243.0	307375.0938887348	0.007911920547485352
5	0.0999999999999992	8295	-22243.8	307376.5316751413	0.005986213684082031
5	0.0999999999999992	8296	-22244.6	307377.9694615478	0.009063005447387695
5	0.0999999999999992	8297	-22245.2	307379.4072479542	0.011208057403564453
5	0.0999999999999992	8298	-22245.8	307380.8450343607	0.011021137237548828
5	0.0999999999999992	8299	-22246.2	307382.28282076714	0.01310586929321289
5	0.0999999999999992	8300	-22246.6	307383.72060717363	0.01204371452331543
5	0.0999999999999992	8301	-22247.0	307385.21162800083	0.06245827674865723
5	0.0999999999999992	8302	-22247.4	307386.702648828	0.01148223876953125
5	0.0999999999999992	8303	-22247.8	307388

...

50	0.0999999999999992	99890	1499317.4	537407.8036183058	0.06682181358337402
50	0.0999999999999992	99891	1499350.6	537410.8380957659	0.12364912033081055
50	0.0999999999999992	99892	1499383.6	537413.872573226	0.06677031517028809
50	0.0999999999999992	99893	1499416.6	537416.9070506862	0.04842948913574219
50	0.0999999999999992	99894	1499449.6	537419.9415281463	0.02797865867614746
50	0.0999999999999992	99895	1499482.6	537422.9760056064	0.031183958053588867
50	0.0999999999999992	99896	1499515.6	537426.0104830664	0.0450739860534668
50	0.0999999999999992	99897	1499548.6	537429.0449605265	0.03980207443237305
50	0.0999999999999992	99898	1499581.6	537432.0794379867	0.03903031349182129
50	0.0999999999999992	99899	1499614.6	537435.1139154468	0.05049443244934082
50	0.0999999999999992	99900	1499647.6	537438.1483929069	0.044922828674316406
50	0.0999999999999992	99901	1499680.6	537441.205191231	0.09647846221923828
50	0.0999999999999992	99902	1499713.6	537444.2619895551	0.08380818367004395
50	0.09999999

50	0.0999999999999992	99997	1502910.4	537746.5579713921	0.037444353103637695
50	0.0999999999999992	99998	1502944.6	537749.8672306912	0.028936147689819336
50	0.0999999999999992	99999	1502978.8	537753.1764899906	0.04172849655151367
50	0.0999999999999992	100000	1503013.0	537756.4857492899	0.038289785385131836
50	0.0999999999999992	100001	1503047.4	537759.8154756709	0.12275958061218262
50	0.0999999999999992	100002	1503081.6	537763.1452020517	0.059838294982910156
50	0.0999999999999992	100003	1503115.8	537766.4749284327	0.059348106384277344
50	0.0999999999999992	100004	1503150.0	537769.8046548136	0.033612728118896484
50	0.0999999999999992	100005	1503184.2	537773.1343811946	0.035863399505615234
50	0.0999999999999992	100006	1503218.4	537776.4641075756	0.03894925117492676
50	0.0999999999999992	100007	1503252.6	537779.7938339565	0.05090188980102539
50	0.0999999999999992	100008	1503286.8	537783.1235603375	0.03629875183105469
50	0.0999999999999992	100009	1503321.0	537786.4532867183	0.0518436431884

50	0.0999999999999992	100103	1506634.4	538111.1607716947	0.06183481216430664
50	0.0999999999999992	100104	1506670.6	538114.7322220457	0.03490710258483887
50	0.0999999999999992	100105	1506706.8	538118.3036723966	0.03390812873840332
50	0.0999999999999992	100106	1506743.0	538121.8751227475	0.03587007522583008
50	0.0999999999999992	100107	1506779.0	538125.4465730984	0.03394365310668945
50	0.0999999999999992	100108	1506815.0	538129.0180234492	0.046842336654663086
50	0.0999999999999992	100109	1506851.0	538132.5894738002	0.07410430908203125
50	0.0999999999999992	100110	1506887.0	538136.1609241511	0.04648089408874512
50	0.0999999999999992	100111	1506923.0	538139.7518779698	0.10768795013427734
50	0.0999999999999992	100112	1506959.0	538143.3428317887	0.06186676025390625
50	0.0999999999999992	100113	1506995.0	538146.9337856075	0.07491183280944824
50	0.0999999999999992	100114	1507031.2	538150.5247394263	0.03644394874572754
50	0.0999999999999992	100115	1507067.4	538154.1156932451	0.0388972759246826

50	0.0999999999999992	100209	1510565.0	538500.3208626868	0.0370028018951416
50	0.0999999999999992	100210	1510603.6	538504.0929305996	0.03789782524108887
50	0.0999999999999992	100211	1510642.2	538507.8879266027	0.11473751068115234
50	0.0999999999999992	100212	1510680.8	538511.6829226059	0.055181026458740234
50	0.0999999999999992	100213	1510719.6	538515.477918609	0.06236910820007324
50	0.0999999999999992	100214	1510758.4	538519.2729146122	0.03493857383728027
50	0.0999999999999992	100215	1510797.4	538523.0679106154	0.03993964195251465
50	0.0999999999999992	100216	1510836.4	538526.8629066185	0.03790712356567383
50	0.0999999999999992	100217	1510875.4	538530.6579026217	0.046842336654663086
50	0.0999999999999992	100218	1510914.4	538534.4528986248	0.04305601119995117
50	0.0999999999999992	100219	1510953.4	538538.247894628	0.0359034538269043
50	0.0999999999999992	100220	1510992.4	538542.0428906311	0.037482500076293945
50	0.0999999999999992	100221	1511031.4	538545.8597892476	0.10442972183227539


50	0.0999999999999992	100315	1514893.4	538912.14683613	0.031951904296875
50	0.0999999999999992	100316	1514936.2	538916.1251690697	0.03552103042602539
50	0.0999999999999992	100317	1514979.2	538920.1035020096	0.03685879707336426
50	0.0999999999999992	100318	1515022.2	538924.0818349494	0.037566184997558594
50	0.0999999999999992	100319	1515065.4	538928.0601678893	0.03197312355041504
50	0.0999999999999992	100320	1515108.6	538932.0385008291	0.03850364685058594
50	0.0999999999999992	100321	1515151.8	538936.0358234482	0.13004136085510254
50	0.0999999999999992	100322	1515195.0	538940.0331460673	0.08550620079040527
50	0.0999999999999992	100323	1515238.4	538944.0304686863	0.04383683204650879
50	0.0999999999999992	100324	1515281.8	538948.0277913053	0.03431057929992676
50	0.0999999999999992	100325	1515325.4	538952.0251139244	0.03793454170227051
50	0.0999999999999992	100326	1515369.0	538956.0224365435	0.05752205848693848
50	0.0999999999999992	100327	1515412.4	538960.0197591626	0.03394126892089844
50

In [57]:
test_env = Environment1(test)
pobs = test_env.reset()
test_acts = []
test_rewards = []

for _ in range(len(test_env.data)-1):
    
    pact = Q(torch.from_numpy(np.array(pobs, dtype=np.float32).reshape(1, -1)))
    pact = np.argmax(pact.data)
    test_acts.append(pact.item())
            
    obs, reward, done = test_env.step(pact.numpy())
    test_rewards.append(reward)

    pobs = obs
        
test_profits = test_env.profits

In [60]:
print("Profits:", test_profits)

Profits: 377.05000000000007


In [ ]:
# For 50 epochs: profits = 377.05000000000007